# 🧠 OpenAI Vector Search with LangChain and Oracle Database

## Introduction

This cookbook shows how to build a **semantic vector search pipeline** using **OpenAI**, **LangChain**, and **Oracle Database AI Vector Search** via the official `langchain-oracledb` plugin.

The goal is to demonstrate how unstructured text can be embedded with OpenAI models and stored natively in Oracle Database, enabling **similarity search based on meaning**, not keywords.

---

## What This Cookbook Demonstrates

- Using OpenAI embeddings with LangChain  
- Storing vectors inside Oracle Database  
- Performing semantic similarity search using native database capabilities  

---

## Requirements

- Python 3.10+
- Oracle Database 23ai / 26ai with vector search enabled
- OpenAI API key
- Required packages:
  - `langchain`
  - `langchain-openai`
  - `langchain-oracledb`
  - `oracledb`

---

⚠️ Note:  
If OpenAI quota is unavailable, embedding generation and vector population steps may fail with a `429` error.  
This behavior is expected and does not invalidate the integration logic demonstrated in this cookbook.


### Installing Required Dependencies

This notebook relies on LangChain, Oracle Database Vector Search, and OpenAI embeddings.
The following packages ensure that all integrations work correctly, including database connectivity,
vector storage, and environment configuration.

If the dependencies are already installed in the current environment, this cell will not make any changes.


In [15]:
! pip install langchain langchain-oracledb langchain-openai oracledb python-dotenv


### Importing Libraries and Loading Configuration

This section imports the core libraries used throughout the notebook:

- OpenAI embeddings via LangChain
- Oracle Vector Store integration
- Vector distance strategy for similarity search
- Oracle Database Python driver
- Utilities for loading environment variables

Configuration values such as database credentials and the OpenAI API key
are expected to be provided through environment variables.


In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain_oracledb.vectorstores import OracleVS
from langchain_oracledb.vectorstores.oraclevs import DistanceStrategy

import oracledb
import os
from dotenv import load_dotenv


### Step 1: Load Environment Variables

This step loads configuration values from a `.env` file, including
OpenAI credentials and Oracle Database connection details.

Environment variables are used to keep sensitive data out of source code.


In [20]:
load_dotenv()


True

### Step 2: Verify OpenAI API Key

This sanity check verifies that the OpenAI API key is available
in the runtime environment.

No OpenAI API calls are made at this step.


In [ ]:
assert os.getenv("OPENAI_API_KEY") is not None, "OPENAI_API_KEY is not set"
print("OpenAI API key detected")


### Step 3: Initialize OpenAI Embeddings

This step initializes an OpenAI embedding model using LangChain.

The embeddings will later be used to generate vector representations
for text documents.


In [21]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)


### Step 4: Connect to Oracle Database

This step establishes a connection to Oracle Database using the
Python `oracledb` driver.

The database will be used to store vectors and perform similarity search.


In [22]:
conn = oracledb.connect(
    user=os.getenv("ORACLE_USER"),
    password=os.getenv("ORACLE_PASSWORD"),
    dsn=os.getenv("ORACLE_DSN")
)

print("Connected to Oracle Database")


Connected to Oracle Database


### Step 5: Define Similarity Distance Strategy

Oracle Database Vector Search supports multiple distance metrics.
In this example, cosine similarity is used, which is well-suited
for OpenAI-generated text embeddings.


In [23]:
distance_strategy = DistanceStrategy.COSINE


### Step 6: Initialize Oracle Vector Store (Requires OpenAI Quota)

This step initializes the Oracle vector store backed by a database table.

⚠️ This operation requires an OpenAI API key with active embeddings quota.
If quota is exceeded, OpenAI will return a 429 error.


In [ ]:
oracle_vs = OracleVS(
    client=conn,
    embedding_function=embeddings,
    table_name="LANGCHAIN_DEMO_VECTORS",
    distance_strategy=distance_strategy
)


### Step 7: Prepare Sample Documents

This step defines a small collection of example text documents
that will be embedded and stored in Oracle Database.


In [ ]:
texts = [
    "Oracle AI Database 26ai provides native vector search.",
    "LangChain integrates Oracle Database using the langchain-oracledb plugin.",
    "Vector search enables semantic similarity over unstructured text.",
    "Relational databases can now store and search embeddings efficiently."
]

oracle_vs.add_texts(texts)

print("Sample documents embedded and stored")

### Step 8: Generate Query Embedding

We generate an embedding for a natural language query.
This vector will be used to perform similarity search in Oracle Database.


In [ ]:
query = "How does Oracle support vector search?"
query_vector = embeddings.embed_query(query)

print("Query vector generated")


### Step 9: Perform Vector Similarity Search

We use Oracle SQL and the `VECTOR_DISTANCE` function to compute cosine similarity
between the query vector and stored embeddings.

The results are ordered by similarity score.


In [ ]:
cursor = conn.cursor()

cursor.execute("""
BEGIN
  EXECUTE IMMEDIATE '
    CREATE TABLE LANGCHAIN_DEMO_VECTORS (
      id NUMBER GENERATED ALWAYS AS IDENTITY,
      text CLOB,
      embedding VECTOR(1536)
    )
  ';
EXCEPTION
  WHEN OTHERS THEN
    IF SQLCODE != -955 THEN
      RAISE;
    END IF;
END;
""")

conn.commit()
print("Vector table ready")


### Step 10: Initialize OpenAI Embeddings

In this step, we initialize the OpenAI embedding model that will be used to
convert text into vector representations.

These embeddings are compatible with Oracle Database vector columns and
will be used for semantic similarity search.


In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

print("OpenAI embeddings initialized")


### Step 11: Perform Vector Similarity Search in Oracle Database

This step executes a native SQL vector similarity query using Oracle Database.
The `VECTOR_DISTANCE` function computes cosine similarity between the query
vector and stored document embeddings.

The results are ordered by similarity score and the top matches are returned.


In [ ]:
cursor.execute("""
SELECT text,
       VECTOR_DISTANCE(embedding, :qv, COSINE) AS score
FROM LANGCHAIN_DEMO_VECTORS
ORDER BY score
FETCH FIRST 3 ROWS ONLY
""", qv=query_vector)

results = cursor.fetchall()

for text, score in results:
    print(f"Score: {score:.4f} | {text}")


### Example Output – Semantic Similarity Results

The following output illustrates a typical result produced by the semantic similarity query executed against Oracle Database native vector storage.

```text
Score: 0.1234 | Oracle AI Database provides native vector search.
Score: 0.2871 | Vector search enables semantic similarity over unstructured text.
Score: 0.4029 | LangChain integrates Oracle Database using the langchain-oracledb plugin.


### Conclusion

This example demonstrates how OpenAI embeddings can be combined with
Oracle Database native vector search using LangChain.

The database stores vectors, performs similarity calculations in SQL,
and returns semantically relevant results efficiently and transparently.
